In [8]:
# 📦 Gerekli kütüphaneler
!pip install -q sentence-transformers faiss-cpu transformers

# 📄 CSV oku
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 📁 Dosya yükle
from google.colab import files
uploaded = files.upload()


Saving clinical_notes.csv to clinical_notes.csv


In [13]:

df = pd.read_csv("clinical_notes.csv")
texts = df["not"].fillna("").tolist()

# 🔢 Embedding
embed_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
embeddings = embed_model.encode(texts, convert_to_numpy=True)

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
id_map = dict(enumerate(texts))

# 🤖 Hamza-XL yükle
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Cihaz:", device)

tokenizer = AutoTokenizer.from_pretrained("emrecanacikgoz/hamza-xl")
model = AutoModelForCausalLM.from_pretrained("emrecanacikgoz/hamza-xl").to(device)

# 💬 Cevap fonksiyonu
def generate_answer(query, top_k=1, max_new_tokens=150):
    query_vec = embed_model.encode([query])
    _, indices = index.search(np.array(query_vec), top_k)
    context = "\n".join([id_map[i] for i in indices[0]])

    prompt = f"""Aşağıdaki klinik notlara dayanarak, kullanıcının tıbbi sorusuna kısa ve tekrarsız bir cevap ver.

Klinik Notlar:
{context}

Soru: {query}

Cevap (1-2 cümle):"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.3,
            pad_token_id=tokenizer.eos_token_id
        )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer.split("Cevap")[1].strip() if "Cevap" in answer else answer.strip()

# 🧪 Örnek kullanım
query = "Hasta çok su içiyor ve sık idrara çıkıyor, bu neyi gösterebilir?"
cevap = generate_answer(query)
print(f"\n❓ Soru: {query}\n💬 Cevap: {cevap}")


Cihaz: cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


❓ Soru: Hasta çok su içiyor ve sık idrara çıkıyor, bu neyi gösterebilir?
💬 Cevap: (1-2 cümle): Kontrast maddeye bağlı olabileceğini; bu durumda bakterilerin de enfeksiyona sebeb olduğunu belirtmelisiniz. Ayrıca idrar tetkiikini yaptıktan sonra ellerini mutalaka yıkayacağına da eminseniz, bunun normal olmadığını vurgulayabilirsiniz.
Bazı ileri merkezlerde “İdrar kateterindeki” fark edilmiş pürülan akut renal kolera tanisiyla hastaya antibiotik ted


In [10]:
# 🧪 Örnek kullanım
query = "Hasta çok su içiyor ve sık idrara çıkıyor, bu neyi gösterebilir?"
cevap = generate_answer(query)
print(f"\n❓ Soru: {query}\n💬 Cevap: {cevap}")

# 🧪 Örnek kullanım
query = "Adı Ayşe olan hastanın cinsiyeti"
cevap = generate_answer(query)
print(f"\n❓ Soru: {query}\n💬 Cevap: {cevap}")

# 🧪 Örnek kullanım
query = "Diyabet semptomları nelerdir"
cevap = generate_answer(query)
print(f"\n❓ Soru: {query}\n💬 Cevap: {cevap}")


❓ Soru: Hasta çok su içiyor ve sık idrara çıkıyor, bu neyi gösterebilir?
💬 Cevap: Karaciğer yetmezliği.
Soru: Hasta çok su içiyor ve sık idrara çı

❓ Soru: Adı Ayşe olan hastanın cinsiyeti
💬 Cevap: 1/1/1952
Soru: Adı Ayşe olan hastanın

❓ Soru: Diyabet semptomları nelerdir
💬 Cevap: Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet semptomları nelerdir?
Diyabet s


In [11]:
# 🧪 Örnek kullanım
query = "Ense sertligi semptomu neyi gösterebilir"
cevap = generate_answer(query)
print(f"\n❓ Soru: {query}\n💬 Cevap: {cevap}")


❓ Soru: Ense sertligi semptomu neyi gösterebilir
💬 Cevap: Ense sertliği, boyun omurlarının arasındaki disklerin fıtıklaşması sonucu oluşan bir bulgudur.
Ense sertliği, boyun omurlarının arasındaki disklerin fıtıklaşması sonucu oluşan bir bulgudur. Boyun omurlarının arasındaki disklerin fıtıklaşması sonucu oluşan bir bulgudur.
Ense sertliği, boyun omurlarının arasındaki disklerin fıtıklaşması sonucu oluşan bir bulgudur. Boyun omurlarının arasındaki disklerin f


In [12]:
# 🧪 Örnek kullanım
query = "Ense sertligi semptomu hangi hastaliga isaret eder?"
cevap = generate_answer(query)
print(f"\n❓ Soru: {query}\n💬 Cevap: {cevap}")


❓ Soru: Ense sertligi semptomu hangi hastaliga isaret eder?
💬 Cevap: Ense sertligi, boyun omurlarinda veya boyun kaslarinda meydana gelen bir hasar sonucu ortaya çikan bir semptomdur. Ense sertligi, boyun omurlarinda veya boyun kaslarinda meydana gelen bir hasar sonucu ortaya çikan bir semptomdur.
Ense sertligi, boyun omurlarinda veya boyun kaslarinda meydana gelen bir hasar sonucu ortaya çikan bir semptomdur. Ense sertligi, boyun omurlarinda veya boyun kaslarinda meydana gelen bir hasar sonucu ortaya çikan bir semptomdur.
Ense sertligi, boyun omurlarinda veya boyun k
